In [1]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Flatten, Dense

from PIL import Image
import numpy as np
import pandas as pd
import os
import warnings

In [2]:
path = '/Users/utkarsh/Desktop/study/iitj/sem2/ml2/assignment2/101_ObjectCategories'
valid_exts = [".jpg", ".gif", ".png", ".jpeg"]
print ("[%d] CATEGORIES ARE IN \n %s" % (len(os.listdir(path)), path))

[10] CATEGORIES ARE IN 
 /Users/utkarsh/Desktop/study/iitj/sem2/ml2/assignment2/101_ObjectCategories


In [3]:
categories = sorted(os.listdir(path))
ncategories = len(categories)
imgs = []
labels = []
cat_dict = {}
# LOAD ALL IMAGES 
for i, category in enumerate(categories):
    iter = 0
    for f in os.listdir(path + "/" + category):
        if iter == 0:
            ext = os.path.splitext(f)[1]
            if ext.lower() not in valid_exts:
                continue
            fullpath = os.path.join(path + "/" + category, f)
            img = load_img(fullpath, target_size=(200, 300))
            img = img_to_array(img)
            img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
            imgs.append(img) # NORMALIZE IMAGE 
            label_curr = i
            labels.append(label_curr)
            cat_dict[category] = i
        #iter = (iter+1)%10;
print ("Num imgs: %d" % (len(imgs)))
print ("Num labels: %d" % (len(labels)) )
print (ncategories)
print(cat_dict)

Num imgs: 60
Num labels: 60
10
{'accordion': 0, 'airplanes': 1, 'anchor': 2, 'ant': 3, 'bass': 4, 'beaver': 5, 'binocular': 6, 'bonsai': 7, 'buddha': 8, 'butterfly': 9}


In [4]:
seed = 7
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size = 0.2) # Change the proportion of training and test

In [7]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
#model_vgg16_conv.summary()

#Create your own input format (here 3x200x200)
ip = Input(shape=(200,300,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(ip)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(1000, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(inputs=ip, outputs=x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 200, 300, 3)]     0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              113250304 
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 1000)              4097000   
Total params: 148,843,304
Trainable params: 148,843,304
Non-trainable params: 0
_____________________________________________

In [8]:
temp_labels = []
temp_features = []

for index in range(0, len(X_train)):
    img = X_train[index]
    img = preprocess_input(img)
    # Extracting features
    yhat = my_model.predict(img)
    # convert the probabilities to class labels
    label = decode_predictions(yhat)
    # retrieve the most likely result, e.g. highest probability
    label = label[0][0]
    # print the classification
    print('%s (%.2f%%)' % (label[1], label[2]*100))

pool_table (89.63%)
acorn (33.88%)
polecat (61.36%)
acorn (90.27%)
medicine_chest (59.50%)
hook (66.87%)
howler_monkey (100.00%)
mitten (92.76%)
oystercatcher (70.69%)
howler_monkey (62.04%)
oystercatcher (99.68%)
howler_monkey (67.06%)
bobsled (98.12%)
wallet (45.95%)
acorn (61.60%)
oystercatcher (81.44%)
ice_cream (95.96%)
oystercatcher (93.72%)
pool_table (59.05%)
agama (29.88%)
echidna (95.71%)
bobsled (99.85%)
howler_monkey (97.34%)
coyote (97.83%)
sea_cucumber (44.24%)
Yorkshire_terrier (99.43%)
howler_monkey (97.19%)
Yorkshire_terrier (70.22%)
bobsled (91.37%)
coho (56.70%)
acorn (68.24%)
monarch (99.07%)
European_gallinule (62.35%)
oystercatcher (96.14%)
howler_monkey (30.06%)
parking_meter (91.08%)
oystercatcher (100.00%)
lotion (76.78%)
Eskimo_dog (53.07%)
Eskimo_dog (88.16%)
cab (84.36%)
papillon (59.77%)
pool_table (99.24%)
howler_monkey (99.50%)
howler_monkey (98.59%)
Yorkshire_terrier (34.80%)
howler_monkey (75.80%)
bobsled (54.46%)
